In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null --fix-missing
!wget https://mirrors.estointernet.in/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xvf spark-3.0.1-bin-hadoop2.7.tgz -qq > /dev/null
!pip install -q findspark
!pip install pyLDAvis textacy -qq > /dev/null

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources 

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
sc = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
!rm -rf topic_centrality/

In [3]:
!pip install pyLDAvis textacy -qq > /dev/null

In [4]:
!git clone https://github.com/AnupJoseph/topic_centrality.git

Cloning into 'topic_centrality'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 494 (delta 8), reused 19 (delta 7), pack-reused 463
Receiving objects: 100% (494/494), 49.49 MiB | 24.21 MiB/s, done.
Resolving deltas: 100% (310/310), done.
Checking out files: 100% (118/118), done.


In [5]:
#spark
from pyspark.sql.types import *
import pyspark.sql.functions as func

# gensim
import gensim
# from gensim.utils import simple_preprocess
# from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import datapath
from gensim import corpora, models

# general
import re

import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [6]:
data = sc.read.format("csv").options(header='true', inferschema='true').load(os.path.realpath("sparkling_clean_data.csv"))

In [7]:
display(data)

DataFrame[_c0: int, Unnamed: 0: int, id: bigint, created_at: string, original_text: string, clean_text: string, retweet_count: string, hashtags: string, mentions: string, original_author: string]

In [9]:
tweets = data.rdd.map(lambda x : x["clean_text"]).filter(lambda x: x is not None)
tweets = tweets.map( lambda document: re.split(" ", document)).zipWithIndex()

In [10]:
processed_docs = tweets.map(lambda x: x[0]).collect()
dict = gensim.corpora.Dictionary(processed_docs)

dict.filter_extremes(no_below=4, no_above=0.8, keep_n=10000)
bow_corpus = [dict.doc2bow(doc) for doc in processed_docs]

In [11]:
bow_doc_1 = bow_corpus[0]
print("Showing bag of words for doc 0")

for i in range(len(bow_doc_1)):
  print(f"Word {bow_doc_1[i][0]} (\"{dict[bow_doc_1[i][0]]}\") appears {bow_doc_1[i][1]} time.")

Showing bag of words for doc 0
Word 0 ("Bernie") appears 1 time.
Word 1 ("right") appears 1 time.


In [12]:
ldaModel = gensim.models.LdaMulticore(
  bow_corpus,
  num_topics=7,
  id2word=dict,
  alpha=0.01,
  eta=0.5,
  offset=8
)

In [13]:
# Compute Perplexity
print('LDA Perplexity: ', ldaModel.log_perplexity(bow_corpus)) # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherenceModelLda = CoherenceModel(model=ldaModel, texts=processed_docs, dictionary=dict, coherence='c_v')
coherenceLda = coherenceModelLda.get_coherence()
print('LDA Coherence Score: ', coherenceLda)

LDA Perplexity:  -7.901940958199533
LDA Coherence Score:  0.2514041013339123


In [14]:

'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in ldaModel.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.011*"Trump" + 0.011*"work" + 0.010*"President" + 0.010*"right" + 0.009*"people" + 0.009*"worker" + 0.008*"thank" + 0.007*"United" + 0.007*"job" + 0.007*"States"


Topic: 1 
Words: 0.014*"great" + 0.010*"Congress" + 0.008*"need" + 0.008*"Trump" + 0.008*"work" + 0.008*"year" + 0.007*"people" + 0.007*"thank" + 0.006*"Iran" + 0.006*"testify"


Topic: 2 
Words: 0.009*"people" + 0.007*"debt" + 0.007*"country" + 0.006*"time" + 0.006*"Democrats" + 0.006*"year" + 0.006*"great" + 0.005*"good" + 0.005*"work" + 0.005*"China"


Topic: 3 
Words: 0.019*"Trump" + 0.013*"great" + 0.007*"country" + 0.007*"vote" + 0.007*"help" + 0.007*"Wall" + 0.007*"year" + 0.006*"need" + 0.006*"Donald" + 0.006*"Puerto"


Topic: 4 
Words: 0.008*"need" + 0.008*"America" + 0.007*"Americans" + 0.007*"country" + 0.007*"Trump" + 0.007*"company" + 0.007*"student" + 0.005*"child" + 0.005*"good" + 0.005*"price"


Topic: 5 
Words: 0.013*"Trump" + 0.012*"people" + 0.009*"vote" + 0.008*"want" + 0.008*"wage" + 0.

In [15]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.gensim.prepare(ldaModel, bow_corpus, dict, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.066004 -0.112100       1        1  19.068917
3     -0.051587  0.123049       2        1  16.513003
1     -0.026413  0.012904       3        1  15.354032
2     -0.111886  0.005375       4        1  13.639479
4      0.059676  0.067102       5        1  13.364299
0      0.124682  0.007202       6        1  12.126302
5     -0.060475 -0.103532       7        1   9.933967, topic_info=        Term         Freq        Total Category  logprob  loglift
207   health  1171.000000  1171.000000  Default  30.0000  30.0000
204     care  1009.000000  1009.000000  Default  29.0000  29.0000
83      vote   663.000000   663.000000  Default  28.0000  28.0000
448     wage   529.000000   529.000000  Default  27.0000  27.0000
156    Trump  2272.000000  2272.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
376   strong    85.546064   521.513648   Topic7  -5.6212   0.5015
187    today    98.138522  1245.635353   Topic7  -5.4839  -0.2318
181  country    97.276380  1582.840952   Topic7  -5.4927  -0.4802
148    fight    83.994572   661.329768   Topic7  -5.6395   0.2457
142     good    78.726440   923.883291   Topic7  -5.7043  -0.1534

[540 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
377       1  0.039867      
377       2  0.019933      
377       3  0.049834      
377       4  0.009967      
377       6  0.009967      
...     ...       ...   ...
57        3  0.232774  year
57        4  0.151011  year
57        5  0.118473  year
57        6  0.103455  year
57        7  0.040047  year

[2539 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 2, 3, 5, 1, 6])

In [16]:
# Predict topic
topics = ldaModel.get_document_topics(bow_corpus, 0.5)

# Display topics
i = 0
for topic in topics[i]:
  print(f"Topic {i + 1}: {topic} Words: {ldaModel.show_topic(topic[0], 10)}")
  i += 1

Topic 1: (3, 0.9710145) Words: [('Trump', 0.019138848), ('great', 0.012746836), ('country', 0.0072474154), ('vote', 0.007105114), ('help', 0.0069738487), ('Wall', 0.006871343), ('year', 0.0067461855), ('need', 0.006420988), ('Donald', 0.006232076), ('Puerto', 0.0061548147)]
